# Understanding the Impact of Image Quality and Distance of Objects to Object Detection Performance

***A reproduction*** 

## Downscale Data

Folder expansion: Some datasets come with multiple subfolders not needed for the purposes of our reproduction study. Example:

```
img
|_dir1 
    |_img1.png
    |_img2.png
    |_img3.png
|_dir2
    |_img1.png
    |_img2.png
    |_img3.png
|_dir3
(...)
```

In [ ]:
from data_processing.expand_folders import expand_folders
expand_folders("../datasets/ECP2dot5D_day_labels_val/ECP2dot5D/day/labels/val")

### Spatial and Amplitudinal Resolution Downsampling


Spatial - 1.42x Downsampling

In [ ]:
from data_processing.expand_dataset import expand_dataset

INPUT_IMAGE_DIR = "../datasets/ECP/day/img/val"
INPUT_LABEL_DIR = "../datasets/ECP2dot5D_day_labels_val/ECP2dot5D/day/labels/val"
DATASET_OUTPUT_DIR = "../datasets/spatially_compressed"
label_values_to_scale = ["imageheight", "imagewidth", "x0", "y0", "x1", "y1"]
OUTPUT_IMG_DIR = f"{DATASET_OUTPUT_DIR}/img"
OUTPUT_LABEL_DIR = f"{DATASET_OUTPUT_DIR}/labels"

expand_dataset(
    input_dir=INPUT_IMAGE_DIR,
    label_dir=INPUT_LABEL_DIR,
    label_values_to_scale=label_values_to_scale,
    output_img_dir=OUTPUT_IMG_DIR,
    output_label_dir=OUTPUT_LABEL_DIR,
    scale_factors=[720.0/1024], #value from paper 
    qp_values=[],
)

Amplitudinal Downsampling

In [ ]:
from data_processing.expand_dataset import expand_dataset

INPUT_IMAGE_DIR = "../datasets/ECP/day/img/val"
INPUT_LABEL_DIR = "../datasets/ECP2dot5D_day_labels_val/ECP2dot5D/day/labels/val"
DATASET_OUTPUT_DIR = "../datasets/eurocity_original_amplitudinally_compressed"
label_values_to_scale = ["imageheight", "imagewidth", "x0", "y0", "x1", "y1"]
OUTPUT_IMG_DIR = f"{DATASET_OUTPUT_DIR}/img"
OUTPUT_LABEL_DIR = f"{DATASET_OUTPUT_DIR}/labels"

qp_values = [16, 24, 34, 38, 46] #values from paper

expand_dataset(
    input_dir=INPUT_IMAGE_DIR,
    label_dir=INPUT_LABEL_DIR,
    label_values_to_scale=label_values_to_scale,
    scale_factors=[],
    output_img_dir=OUTPUT_IMG_DIR,
    output_label_dir=OUTPUT_LABEL_DIR,
    qp_values=qp_values, #values from paper
    expansion = "amplitudinal"
)

Mixed Downsampling - Combined Set of both

In [ ]:
from data_processing.expand_dataset import expand_dataset

INPUT_IMAGE_DIR = "../datasets/ECP/day/img/val"
INPUT_LABEL_DIR = "../datasets/ECP2dot5D_day_labels_val/ECP2dot5D/day/labels/val"
DATASET_OUTPUT_DIR = "../datasets/mixed"
label_values_to_scale = ["imageheight", "imagewidth", "x0", "y0", "x1", "y1"]
OUTPUT_IMG_DIR = f"{DATASET_OUTPUT_DIR}/img"
OUTPUT_LABEL_DIR = f"{DATASET_OUTPUT_DIR}/labels"
qp_values = [16, 24, 34, 38, 46] #values from paper

expand_dataset(
    input_dir=INPUT_IMAGE_DIR,
    label_dir=INPUT_LABEL_DIR,
    label_values_to_scale=label_values_to_scale,
    output_img_dir=OUTPUT_IMG_DIR,
    output_label_dir=OUTPUT_LABEL_DIR,
    scale_factors=[1, 720.0/1024, 854.0/ 1920], #values from paper
    qp_values=[0, 5, 10, 16, 20, 24, 28, 34, 38, 42, 46, 51], #values from paper
    expansion="mixed", 
    subsample_spatial=True,
    subsample_amplitudinal=True,
)

## Train/Load Model

#### Train

In [1]:
import sys
import os
import torch
import argparse
from pathlib import Path
yolov5_dir = os.path.abspath('yolov5')
if yolov5_dir not in sys.path:
    sys.path.append(yolov5_dir)
from yolov5.train import main, Callbacks
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device = 'cuda'
print(torch.cuda.is_available())
cfg = "ra_yolo5l.yaml"  
opt = argparse.Namespace(
    weights='yolov5l.pt',  # Model weights
    cfg=cfg,  # Empty to use weights' default config
    data=os.path.abspath('../datasets/mixed/data.yaml'),  # Absolute path to dataset
    hyp=os.path.join(yolov5_dir, 'data/hyps/hyp.scratch-low.yaml'),  # Hyperparameters
    epochs=1,
    batch_size=1,
    imgsz=800,  # Fixed from invalid imgsz=4
    rect=False,
    resume=False,
    nosave=False,
    noval=False,
    noautoanchor=False,
    noplots=False,
    evolve=None,
    evolve_population=os.path.join(yolov5_dir, 'data/hyps'),
    resume_evolve=None,
    bucket='',
    cache=None,
    image_weights=False,
    device=device,
    multi_scale=False,
    single_cls=False,
    optimizer='SGD',
    sync_bn=False,
    workers=8,
    project=os.path.join(yolov5_dir, 'runs/train'),
    name='exp',
    exist_ok=True,
    quad=False,
    cos_lr=False,
    label_smoothing=0.0,
    patience=100,
    freeze=[0],  # Freeze first 10 layers
    save_period=-1,
    seed=0,
    local_rank=-1,
    ra_yolo=True,
    entity=None,
    upload_dataset=False,
    bbox_interval=-1,
    artifact_alias='latest',
    ndjson_console=False,
    ndjson_file=False

)

main(opt, callbacks=Callbacks())

train: weights=yolov5l.pt, cfg=ra_yolo5l.yaml, data=/home/vasil/Desktop/frmdl/datasets/mixed/data.yaml, hyp=/home/vasil/Desktop/frmdl/frmdl-object-detection-distance-resolution/yolov5/data/hyps/hyp.scratch-low.yaml, epochs=1, batch_size=1, imgsz=800, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, evolve_population=/home/vasil/Desktop/frmdl/frmdl-object-detection-distance-resolution/yolov5/data/hyps, resume_evolve=None, bucket=, cache=None, image_weights=False, device=cuda, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=/home/vasil/Desktop/frmdl/frmdl-object-detection-distance-resolution/yolov5/runs/train, name=exp, exist_ok=True, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, seed=0, local_rank=-1, ra_yolo=True, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest, ndjson_console=False, ndjson_file=False


True


github: ⚠️ YOLOv5 is out of date by 1 commit. Use 'git pull ultralytics master' or 'git clone https://github.com/ultralytics/yolov5' to update.


AssertionError: Invalid CUDA '--device cuda' requested, use '--device cpu' or pass valid CUDA device(s)

#### Extract results

In [ ]:
import torch 
from yolov5.val import run  as yolov5_run_val

data = '../datasets/mixed/data.yaml'  # Path to your dataset YAML file
weights = 'yolov5n.pt'      # Path to your model weights file
batch_size = 16             # Batch size for validation
imgsz = 4                # Image size for inference
device = 'cuda' if torch.cuda.is_available() else 'cpu'  # Automatically select device

# Run the validation
results, maps, times = yolov5_run_val(
    data=data,              # Dataset configuration
    weights=weights,        # Model weights
    batch_size=batch_size,  # Batch size
    imgsz=imgsz,            # Image size
    device=device,          # Device to run on
    task='val',             # Task type: validation
    save_txt=True,         # Don’t save results to text files
    save_json=True,        # Don’t save results to JSON
    plots=True,             # Generate plots (saved to runs/val/)
)

# Extract metrics from r
mp, mr, map50, map, box_loss, obj_loss, cls_loss = results  # Mean Precision, Mean Recall, mAP@0.5, mAP@0.5:0.95
print(f'Mean Precision: {mp:.4f}')
print(f'Mean Recall: {mr:.4f}')
print(f'mAP@0.5: {map50:.4f}')
print(f'mAP@0.5:0.95: {map:.4f}')

In [ ]:
predictions, im_names, label_names = extract_predictions_from_run(json_path="runs/val/exp/predictions.json",)

## Visualizing Results

#### Reproducing figures from the paper

#### Figure 5

In [ ]:
from visualization.xy_lineplot import basic_lineplot, multi_lineplot

data_dict = {
    "EuroCity Original -> Finetuned Model": ([0, 1, 2, 3, 4, 5], [0, 1, 4, 9, 16, 25]),
    "EuroCity 1.42 -> Finetuned Model": ([0, 1, 2, 6, 8, 9], [0, 1, 5, 10, 16, 27]),
    "EuroCity Original": ([0, 2, 3, 5, 7, 9], [0, 1, 2, 3, 12, 20]), 
    "EuroCity 1.42": ([0, 2, 2, 4, 8, 9], [0, 1, 4, 9, 16, 25])
}
## Figure 5 in the paper 
multi_lineplot(data_dict, xlabel="Megabytes/Image", ylabel="map@50 - All Category")

In [ ]:
from visualization.xy_lineplot import basic_lineplot, multi_lineplot

data_dict = {
    "EuroCity Original": ([0, 1, 2, 3, 4, 5], [25, 16, 10, 7, 3, 1]),
    "EuroCity 1.42 -> Finetuned Model": ([0, 1, 2, 6, 8, 9], [24, 13, 10, 5, 1, 0]),
}
## Figure 5 in the paper 
multi_lineplot(data_dict, xlabel="Distance(m)", ylabel="Recall-Pedestrian")